In [1]:
import sys
sys.path.append('/workspace/Documents')
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import cv2
import nibabel as nb

import PSF_generation.python.functions as ff

In [14]:
# func: convert case filename:
def convert_case_filename(case_filename, from_num_to_name = True, ground_truth_label = None):
    if from_num_to_name == True:
        c = case_filename.split('.')[0]
        if c[-2:] == '00':
            position = 'lcc'
        elif c[-2:] == '01':
            position = 'lmlo'
        elif c[-2:] == '10':
            position = 'rcc'
        elif c[-2:] == '11':
            position = 'rmlo'

        ground_truth_label = [1 if c[-3] == '1' else 0][0]

        case_original_name = '2d_proc.' + c[:-3] + '.' + position + '.dcm'
        return case_original_name, ground_truth_label
    
    if from_num_to_name == False:
        assert ground_truth_label != None, 'ground_truth_label must be provided'
        c = case_filename.split('.')[1]
        if ground_truth_label == 1:
            c = c+'1'
        else:
            c = c+'0'

        position = case_filename.split('.')[2]
        if position == 'lcc':
            c = c+'00'
        elif position == 'lmlo':
            c = c+'01'
        elif position == 'rcc':
            c = c+'10'
        elif position == 'rmlo':
            c = c+'11'
        return c, ground_truth_label

In [2]:
main_path = '/mnt/BPM_NAS/cchen/'
ground_truth_path = os.path.join(main_path,'Dataset994_blur/labelsTs')
predict_path = os.path.join(main_path, 'project_results/nnUNet_results/Dataset994_blur/nnUNetTrainer__nnUNetPlans__2d/fold_0/inference_Dataset994_imagesTs')

In [3]:
blur_cases = ff.find_all_target_files(['*.nii.gz'], ground_truth_path)
print('Total number of cases: ', len(blur_cases))

Total number of cases:  293


# quantitative 

In [16]:
result = []
for i in range(len(blur_cases)):
    case_filename = os.path.basename(blur_cases[i]).split('_')[1]
    case_original_name, ground_truth_label = convert_case_filename(case_filename, from_num_to_name = True)
    
    blur_case = nb.load(blur_cases[i]).get_fdata()
    predict_case = nb.load(os.path.join(predict_path, os.path.basename(blur_cases[i]))).get_fdata()
    dice = ff.np_categorical_dice(predict_case, blur_case,1)
    print('case original name: ', case_original_name, ' label: ', ground_truth_label, ' dice: ', dice)

    # also count the pixel number
    pixel_num_gt = np.sum(blur_case)
    pixel_num_pred = np.sum(predict_case)

    result.append([case_filename, case_original_name, ground_truth_label, dice, pixel_num_gt, pixel_num_pred])

result = pd.DataFrame(result, columns = ['case_new_name', 'case_original_name', 'ground_truth_label', 'dice', 'pixel_num_gt', 'pixel_num_pred'])
result.to_excel(os.path.join(main_path, 'project_results/nnUNet_results/Dataset994_blur/nnUNetTrainer__nnUNetPlans__2d/fold_0/quantitative.xlsx'), index = False)

    

case original name:  2d_proc.0000452316.lcc.dcm  label:  0  dice:  2.3827337580952812e-14
case original name:  2d_proc.0000452316.lmlo.dcm  label:  0  dice:  2.690291008778347e-14
case original name:  2d_proc.0000452316.rcc.dcm  label:  0  dice:  2.4545358595415796e-14
case original name:  2d_proc.0000452316.rmlo.dcm  label:  0  dice:  1.137074307805886e-13
case original name:  2d_proc.0000452317.rcc.dcm  label:  0  dice:  7.054454747083639e-15
case original name:  2d_proc.0000452317.rmlo.dcm  label:  0  dice:  1.4988211771441536e-14
case original name:  2d_proc.0000452317.rcc.dcm  label:  1  dice:  0.762743735119126
case original name:  2d_proc.0000452317.rmlo.dcm  label:  1  dice:  0.8444653145319989
case original name:  2d_proc.0000452321.lcc.dcm  label:  0  dice:  7.45378652354841e-13
case original name:  2d_proc.0000452321.lmlo.dcm  label:  0  dice:  8.305916886012019e-15
case original name:  2d_proc.0000452321.rcc.dcm  label:  0  dice:  1.0325245224563422e-12
case original name: 

In [7]:
result = []
for i in range(0,blur_cases.shape[0]):
    blur_case = blur_cases[i]
    gt = nb.load(blur_case).get_fdata(); gt = np.round(gt)
    pred = nb.load(os.path.join(predict_path, os.path.basename(blur_case))).get_fdata(); pred = np.round(pred)
   
    # dice = ff.np_categorical_dice(pred,gt,target_class = 1)

    gt = [1 if np.sum(gt) == gt.shape[0]*gt.shape[1] else 0][0]
    pred = [1 if np.sum(pred) == pred.shape[0]*pred.shape[1] else 0][0]

    if gt == pred:
        classification = 1
    else:
        classification = 0

    result.append([os.path.basename(blur_case), gt, pred, classification])
    
result = pd.DataFrame(result, columns = ['Case', 'GT', 'Pred', 'Result'])
result.to_excel(os.path.join(main_path, 'project_results/nnUNet_results/Dataset993_blur/nnUNetTrainer__nnUNetPlans__2d/fold_0/classification.xlsx'), index = False)
# result = pd.DataFrame(result, columns = ['Case', 'Dice'])
# result.to_excel(os.path.join(main_path, 'project_results/nnUNet_results/Dataset993_blur/nnUNetTrainer__nnUNetPlans__2d/fold_0/dice.xlsx'), index = False)

In [8]:
predict_blur_cases = ff.find_all_target_files(['*.nii.gz'], predict_path)
real_blur_cases = []
for i in range(0, predict_blur_cases.shape[0]):
    c = predict_blur_cases[i]

    basename = os.path.basename(c)

    not_in = True
    for j in range(0, blur_cases.shape[0]):
        if basename not in blur_cases[j]:
            real_blur_cases.append(blur_cases[j])
print('Total number of cases: ', len(real_blur_cases))

Total number of cases:  125312
